### The MineSweeper Game as a Jupyter Noteook

Minesweeper, the funnest game Windows 3.1 had to offer!

You are presented with a field that contains an unknown number of mines.
You must try to locate all of the mines without blowing yourself up.

At the start of the game, the you are presented a 2D grid [rows x columns] of 
blank cells. You can select toggle between PROBE or MARK mode with the button.

The action that occurs when PROBING depending on what's there:

- When you PROBE a MINE - Kaboom! The game's over. The whole board is revealed.

- For any other cell, PROBING shows the number of cells that have one or more mines
  as an immediate neighbor (including diagonals); the total number of adjacent mines.

- If you probe a cell that has zero adjacent mines, it will be revealed as blank and
  then the game will auto-click all neighboring cells to reveal them as well; since
  the game knows that every cell surrounding is safe to click, we save you the hassle.

Use the MARKING mode to add (or remove) **Check Marks** to cells that you suspect contain mines. 

The game is won when you've **CORRECTLY** marked all cells that contain mines.
If one of your marks is incorrect, then the game will not end.

**Good luck!**


In [8]:
debugging = False
debugging = True

logging = True

def dbg(f, *args):
    if debugging:
        print(('  DBG:' + f).format(*args))

def log(f, *args):
    if logging:
        print((f).format(*args))
        
def logError(f, *args):
    if logging:
        print(('*** ERROR:' + f).format(*args))
        
def className(instance):
    return type(instance).__name__

In [16]:
import ipywidgets as widgets
from IPython.display import display

# UI Abstraction Classes
#
class UI_base():

    def __init__(self, description=None, tooltip=None):
        self.active_description=description
        self.inactive_description=None
        if description:
            self.ui.description=description
        if tooltip != None:
            self.ui.tooltip=tooltip
    
    def active(self, state=True):
        self.ui.disabled=not state
        if self.inactive_description:
            self.ui.description = self.active_description if state else self.inactive_description

    def inactive(self):
        self.active(False)

    def describe(self, description=None, icon=None, style=None):
        ui = self.ui
        if description != None: ui.description=description
        if icon != None: ui.icon=icon
        if style != None: ui.button_style=style    

    @property
    def value(self):
        return self.ui.value
    
    @property
    def width(self):
        return self.ui.layout.width
    
    @width.setter
    def width(self, value):
        self.ui.layout.width = value
        
    def display(self):
        ''' Display the UI element(s) using the underlying framework '''
        ui = self.ui
        display(ui)
    
    def handle(self, callback, parent=None, model=None):
        ''' Sets a refereed object UI click handler '''
        self.callback = callback
        self.ui.value = ( parent, model )       
        self.ui.on_click(self.handler)

    def handler(self, rawui_object):
        ''' The internal handler called by the object event '''
        parent, model = rawui_object.value
        self.callback(self, parent, model)
      
        
class UI_button(UI_base):
    def __init__(self, description=None, tooltip=None, icon=None, width=None, button_style=None):
        self.ui = ui = widgets.Button()
        super(UI_button, self).__init__(description=description, tooltip=tooltip)
        if button_style:
            ui.button_style=button_style # 'success', 'info', 'warning', 'danger' or ''
        if width != None:
            ui.width=width
        if icon != None:
            # Icons names are from https://fontawesome.com/icons?d=gallery&m=free
            ui.icon=icon
        ui.disabled=False       

class UI_slider(UI_base):
    def __init__(self, description=None, min=0, max=9, value=None, tooltip=None, step=1):
        self.ui = ui = ui = widgets.IntSlider()
        super(UI_slider, self).__init__(description=description, tooltip=tooltip)
        ui.min = min
        ui.max = max
        ui.step = step
        ui.value = min if value == None else value

class UI_container(UI_base):
    ''' A UI container is used to create a horizontal or vertical
        flowlayout list of other UI objects (including containers).
    '''   
    def __init__(self, vertical=False):
        self.container = []
        self.vertical = vertical
        self.ui = widgets.VBox() if vertical else widgets.HBox()
        self.ui.children = self.container
        super(UI_container, self).__init__()
       
    def append(self, ui_object):
        ''' Append the raw (non-abstracted) UI object to the container '''
        self.ui.children += (ui_object.ui, )
        self.container.append(ui_object.ui)

    def setlast(self, ui_object):
        ''' Set the last raw (non-abstracted) UI object in the container '''
        self.container[-1] = ui_object.ui
        # May cause a lot of flippy redraws in the UI has lots of items
        self.ui.children = tuple(self.container)

    def __getitem__(self, index):
        return self.container[index]
            

In [17]:
class Cell():
    
    def __init__(self, row, col, rows, cols):
        self.row  = row
        self.col  = col
        self.rows = rows
        self.cols = cols
        self.mine = False
        self.revealed = False
        
    def pos(self):
        return (self.row, self.col)

    def valid_pos(self, row, col):
        return col>=0 and col<self.cols and row>=0 and row<self.rows
    
    def is_valid(self):
        row, col = self.pos()
        return self.valid_pos(row, col)

    def desc(self):
        row, col = self.pos()
        return '{0}{1},{2}'.format('**' if self.mine else '', row, col)
    
    def surroundings(self):
        ''' Return a list the (up to) nine valid positions around this cell '''
        row, col = self.pos()
        for y in range(row-1, row+2):
            for x in range(col-1, col+2):
                if self.valid_pos(y, x):
                    yield (y, x)

In [29]:
from random import randint

UNKNOWN = '?'
EMPTY = ' '
MINE = '**'

class Board():
    def __init__(self, game, rows, cols, density=10):
        '''
        The density is the integer percent of cells which will probably
        contain mines.
        '''
        assert density > 0 and density < 90
        assert cols < 50 and rows < 50

        self.rows = rows
        self.cols = cols
        self.game = game
        self.density = density
        
        columns = []
        vbox = UI_container(vertical=True)
        for y in range(rows):
            row = UI_container()
            for x in range(cols):
                btn = UI_button(description=UNKNOWN, icon='none')
                btn.ui.layout.width = '100px' if debugging else '50px'
                row.append(btn)
                row[x].on_click(self.on_cell_button_clicked_event)
                row[x].value = Cell(y, x, rows, cols)
            vbox.append(row)
            columns.append(row.container)      
        self.board = columns
        self.plant_landmines()
        self.count_all_cells()
        self.uicontainer = vbox

    def button(self, pos):
        ''' Get the button that describes this positon '''
        row, col = pos
        return self.board[row][col]
    
    def cell(self, pos):
        ''' Get the cell that describes this positon '''
        button = self.button(pos)
        return button.value
        
    def all_cell_positions(self):
        ''' Generate a list of all cells on the board '''
        for row in range(self.rows):
            for col in range(self.cols):
                yield (row, col)
                
    def plant_landmines(self):
        ''' Mark a random set of cells as having been mined. '''
        mines = 0
        self.mines_marked = 0
        self.mines_marked_correctly = 0
        for pos in self.all_cell_positions():
            cell = self.cell(pos)
            cell.mine = randint(0,99) < self.density
            mines += 1 if cell.mine else 0
        if mines == 0:
            # Always plant at least one mine.
            row = randint(0, self.rows-1)
            col = randint(0, self.cols-1)
            self.cell((row, col)).mine = True
            mines = 1
        self.mines = mines

    def count_mines(self, button):
        cell = button.value
        mine_count = 1 if cell.mine else 0
        for rc in cell.surroundings():
            c = self.cell(rc)
            mine_count += 1 if c.mine else 0
        cell.mine_count = mine_count
        cell.revealed = False
        
    def count_all_cells(self):
        ''' Compute the adjacent mines for all cells '''
        for pos in self.all_cell_positions():
            button = self.button(pos)
            self.count_mines(button)
        
    def show_count(self, button, cell):
        ''' Show the count all the mines adjacent to this cell '''
        button.description = EMPTY if cell.mine_count == 0 else str(cell.mine_count)
        if cell.mine:
            button.description = MINE
        else:
            if button.icon == 'check':
                button.icon = 'exclamation-circle'
        cell.revealed = True

    def reveal_empty_cells(self, button, cell):
        ''' Flow the revealed cells outward from this one '''
        if cell.mine_count == 0:
            for rc in cell.surroundings():
                c = self.cell(rc)
                b = self.button(rc)
                if not c.revealed:
                    self.show_count(b, c)
                    self.reveal_empty_cells(b, c)
                    
    def reveal_all_cells(self):
        for pos in self.all_cell_positions():
            button = self.button(pos)
            cell = self.cell(pos)
            self.show_count(button, cell)            
            
    def click_button(self, button, cell):
        ''' Do everything related to a single button click on a cell '''
        if self.game.probing:
            button.icon = 'none'
            self.show_count(button, cell)
            if cell.mine:
                self.game.game_over_man()
                self.reveal_all_cells()
            self.reveal_empty_cells(button, cell)
        else:
            if not cell.revealed:
                if button.icon == 'check':
                    # Uncheck a cell that may not really be a mine.
                    button.icon = 'none'
                    self.mines_marked_correctly -= 1 if cell.mine else 0
                    self.mines_marked -= 1
                    if debugging or cell.revealed:
                        self.show_count(button, cell)
                    else:
                        button.description = UNKNOWN
                else:
                    button.icon = 'check'
                    button.description = EMPTY
                    self.mines_marked += 1                   
                    self.mines_marked_correctly += 1 if cell.mine else 0
                dbg("Marked {0} of {1} mines correctly", self.mines_marked_correctly, self.mines)               
                if self.mines_marked_correctly == self.mines and self.mines_marked_correctly == self.mines_marked:
                    self.game.congratulations()
                    self.reveal_all_cells()
        
    def on_cell_button_clicked_event(self, button):
        ''' The cell clicked event handler '''
        cell = button.value
        self.click_button(button, cell)        

In [30]:
MSG_PROBING = 'Currently in Mine PROBING mode. Be Very Careful!'
MSG_MARKING = 'Currently in Mine MARKING mode.'

class Game():
   
    def __init__(self, rows=9, cols=9, density=20):
        self.probing = False
        self.board = None
        self.gameover = True

        gameui = UI_container(vertical=True)

        startButton = UI_button('Start NEW Game')
        startButton.handle(self.start_game_button_event)
        self.startButton = startButton
              
        properties = UI_container()
        self.rowsui = UI_slider('Rows', 2, 10, rows)
        self.colsui = UI_slider('Columns', 2, 10, cols)
        self.denseui = UI_slider('Density', 2, 99, density)
        self.rowsui.ui.tooltip= "Selects the number of rows on the board when you start a new game."
        self.colsui.ui.tooltip = "Selects the number of columns on the board when you start a new game."
        self.denseui.ui.help = "Selects the mine density as an integer percentage when you start a new game."
        properties.append(self.rowsui)
        properties.append(self.colsui)
        properties.append(self.denseui)
        self.properties = properties
                              
        modeButton = UI_button(
            description='Initializing...',
            button_style='info', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Description',
            width='300px',
            icon='check'
        )
                             
        modeButton.handle(self.toggle_mode_button_event)
        modeButton.width = '400px'
        modeButton.inactive()
        self.modeButton = modeButton        
        
        gameui.append(self.startButton)
        gameui.append(self.properties)
        gameui.append(self.modeButton)
        self.gameui = gameui

    def set_running_ui(self):
        running = not self.gameover        
        self.modeButton.active(running)
        self.rowsui.active(not running)
        self.colsui.active(not running)
        self.denseui.active(not running)
        self.startButton.describe('Kill Current Game' if running else 'Start NEW Game')
        
    def start_game_button_event(self, button, parent=None, model=None):        
        self.cols = self.colsui.value
        self.rows = self.rowsui.value
        self.density = self.denseui.value
        
        if self.gameover:
            if self.board == None:
                self.board = Board(self, self.rows, self.cols, self.density)
                self.gameui.append(self.board.uicontainer)
            else:
                # Create and display a new board.
                self.board = Board(self, self.rows, self.cols, self.density)
                self.gameui.setlast(self.board.uicontainer)
            self.gameover = False                
        else:
            self.announce("Game terminated for no good reason by user.")
            self.board.reveal_all_cells()
            self.gameover = True            
        self.set_running_ui()       

    def toggle_mode_button_event(self, button, parent=None, model=None):
        self.probing = not self.probing
        if self.probing:
            button.describe(MSG_PROBING, style='danger', icon='none')
        else:
            button.describe(MSG_MARKING, style='info', icon='check')
            
    def announce(self, f, *args):
        ''' Display a status text line '''
        print((f).format(*args))
        
    def game_over_man(self):
        self.gameover = True
        self.announce("***** KABOOM! *****   You are dead. ")
        self.set_running_ui()

    def congratulations(self):
        if self.gameover:
            self.announce("I hope you're happy now. You've marked every deadly mine correctly.")
            self.announce("Unfortunately, that seems somewhat pointless. Have a nice day.")              
        else:
            self.gameover = True 
            self.announce("Congratulations! You've marked every deadly mine correctly.")
            self.announce("You are a winner!")  
        self.set_running_ui()

    def run(self):
        #self.toggle_mode_button_event(self.modeButton.ui)
        self.toggle_mode_button_event(self.modeButton, self)
        self.announce("Minesweeper! The exciting game of blowing yourself up --- or not!")
        self.set_running_ui()
        self.gameui.display()
        self.announce("The game is won when you've correctly marked every mine.")


In [31]:
debugging = False
Game(5, 4).run()


Minesweeper! The exciting game of blowing yourself up --- or not!


A Jupyter Widget

The game is won when you've correctly marked every mine.
***** KABOOM! *****   You are dead. 
***** KABOOM! *****   You are dead. 
